In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn import datasets, linear_model
import hdbscan
import seaborn as sns
import statsmodels.api as sm
from numpy import linalg
from math import sqrt
import time
from sklearn.decomposition import PCA
from skopt import gp_minimize, dump, load
from skopt.space import Real, Integer
from skopt.plots import plot_convergence
from skopt.utils import use_named_args
%matplotlib inline

In [82]:
dataset = pd.read_csv('shopitem.csv')
Shopgroup = pd.read_csv('Shopgroup.csv')
Itemgroup = pd.read_csv('Itemgroup.csv')

In [83]:
Shopgroup.columns=['matchId','In','Ex']
Itemgroup.columns=['playerId','In','Ex']

In [84]:
sparsity=1-dataset.shape[0]/214200/60
sparsity

0.9669993775287893

In [85]:
Ntr=int(dataset.shape[0]*0.6)
Nv=int(dataset.shape[0]*0.8)
sim_data=dataset.values
n=22170
m=60
train=sim_data[:Ntr,:]
validation=sim_data[Ntr:Nv,:]
test=sim_data[Nv:,:]
x_train=train[:,:2]  
x_test=test[:,:2]
x_valid=validation[:,:2]
x_train=x_train.astype(int)
x_test=x_test.astype(int)
x_valid=x_valid.astype(int)
y=train[:,2]
y_test=test[:,2]
y_valid=validation[:,2]

In [86]:
#player set
ind1=[] #matched played for each user
y1=[]
#match set
ind2=[] #players in this match
y2=[]
for u in range(n):
    ind1.append(x_train[x_train[:,0]==u,1])
    y1.append(train[x_train[:,0]==u,2])
for i in range(m):
    ind2.append(x_train[x_train[:,1]==i,0])
    y2.append(train[x_train[:,1]==i,2])

In [87]:
def shrink(x,l):
    if x>l/2:
        X=x-l/2
    elif x<-l/2:
        X=x+l/2
    else:
        X=0
    return X
Vshrink= np.vectorize(shrink)

In [88]:
def mylasso(y,x,k,l,L):
    betaols=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = Vshrink(betaols,l)
    return beta

In [89]:
def myl0(y,x,k,l,L):
    beta=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = beta*((beta>np.median(beta))+1)
    return beta

In [90]:
def mygsm(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb): 
    inM=matchgroup.In.max()
    exM=matchgroup.Ex.max()
    inN=playergroup.In.max()
    exN=playergroup.Ex.max()
    P=np.random.normal(0,0.1,(n,k))
    Q=np.random.normal(0,0.1,(m,k))
    S=np.random.normal(0,0.1,(n,k))
    T=np.random.normal(0,0.1,(m,k))
    A=np.random.normal(0,0.1,(n,k))
    B=np.random.normal(0,0.1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    Anew=np.zeros(shape=(n,k))
    Bnew=np.zeros(shape=(m,k))
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew+Snew+Anew-P-S-A,Pnew+Snew+Anew-P-S-A))/n/k+np.sum(np.multiply(Qnew+Tnew+Bnew-Q-T-B,Qnew+Tnew+Bnew-Q-T-B))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:])@(T[i,:]+B[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)
            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+B[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)
            for i in range(exM):
                I=matchgroup.matchId[matchgroup.Ex==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+Tnew[j,:])
                    r=np.concatenate([r,R])
                Bnew[I,:]=foo(r,xpsa,k,lb,Lb)
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k+np.sum(np.multiply(Bnew-B,Bnew-B))/exM/k
            Q=Qnew
            T=Tnew
            B=Bnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:])@(S[i,:]+A[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)

            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+A[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)

            for i in range(exN):
                I=playergroup.playerId[playergroup.Ex==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+Snew[j,:])
                    r=np.concatenate([r,R])
                Anew[I,:]=foo(r,xqtb,k,la,La)  
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k+np.sum(np.multiply(Anew-A,Anew-A))/exN/k
            P=Pnew
            S=Snew
            A=Anew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]+A[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:]+B[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

In [91]:
def mygsm1(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,lq,lt,Lp,Ls,Lq,Lt): 
    inM=matchgroup.In.max()
    inN=playergroup.In.max()
    P=np.random.normal(0,0.1,(n,k))
    Q=np.random.normal(0,0.1,(m,k))
    S=np.random.normal(0,0.1,(n,k))
    T=np.random.normal(0,0.1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]+S[x_train[:,0],:]),(Q[x_train[:,1],:]+T[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew+Snew-P-S,Pnew+Snew-P-S))/n/k+np.sum(np.multiply(Qnew+Tnew-Q-T,Qnew+Tnew-Q-T))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:])@(T[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)
            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:])@(Qnew[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k
            Q=Qnew
            T=Tnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:])@(S[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)
            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:])@(Pnew[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k
            P=Pnew
            S=Snew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

In [92]:
def mygsm0(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,n,m,k,lp,lq,Lp,Lq): 
    P=np.random.normal(2,1,(n,k))
    Q=np.random.normal(2,1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]),(Q[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]
                r=y2[i]
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)   
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
            Q=Qnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]
                r=y1[i]
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
            P=Pnew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]),(Q[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

2-layered gssvd rmse focused k =2,4,6,8,10

In [93]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
la = Real(low=1e-7, high=1, prior='uniform',
                             name='la')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
lb = Real(low=1e-7, high=1, prior='uniform',
                             name='lb')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
La = Real(low=1e-7, high=1, prior='uniform',
                             name='La')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
Lb = Real(low=1e-6, high=1, prior='uniform',
                             name='Lb')
dimensions = [lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb]

In [94]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,3*k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [95]:
start=time.time()
rmset=np.zeros(shape=(3,5))
rmsev=np.zeros(shape=(3,5))
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmsev[2,k-1]=search_result.fun
    rmset[2,k-1]=mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,3*k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7],
    para[8],
    para[9],
    para[10],
    para[11])[0]
print(time.time()-start)


lp: 0.001



E:\anaconda\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index



rmse: 3.2239348227718465

lp: 0.6144175091316652


rmse: 2.521197963456694

lp: 0.6462116670550124


rmse: 2.801414759189804

lp: 0.1514216381050598


rmse: 2.619057110573792

lp: 0.11198669533922999


rmse: 2.7995408165170854

lp: 0.011956015563274863


rmse: 2.6469607345629718

lp: 0.8408727554016742


rmse: 2.7849153148211423

lp: 0.5530272798460709


rmse: 2.4638109583755283

lp: 0.6798611433280427


rmse: 2.8068984079772163

lp: 0.8399636654987769


rmse: 2.8095601268594015

lp: 0.18071175830946865


rmse: 2.7855279104825335

lp: 1.0



E:\anaconda\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index



rmse: 2.51848060967039

lp: 1.0



...


rmse: 2.627162715669281

lp: 0.7533150772157009



E:\anaconda\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index



rmse: 2.142844354434309



E:\anaconda\lib\site-packages\ipykernel_launcher.py:63: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


7849.463581562042


1-layered gssvd rmse focused 

In [96]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
dimensions = [lp,ls,lq,lt,Lp,Ls,Lq,Lt]

In [97]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,lq,lt,Lp,Ls,Lq,Lt):
    rmse,yhat = mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,3*k,lp,ls,lq,lt,Lp,Ls,Lq,Lt)
    return rmse

In [98]:
start=time.time()
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=70,
                            x0=default_parameters)
    para=search_result.x
    rmsev[1,k-1]=search_result.fun
    rmset[1,k-1]=mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,3*k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7])[0]
print(time.time()-start)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
E:\anaconda\lib\site-packages\ipykernel_

4247.410177230835


nogroupsvd rmse focused

In [99]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
dimensions = [lp,lq,Lp,Lq]

In [100]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,lq,Lp,Lq):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,
                      n,m,3*k,lp,lq,Lp,Lq)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [101]:
start=time.time()
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmsev[0,k-1]=search_result.fun
    rmset[0,k-1]=mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,n,m,3*k,para[0],
    para[1],
    para[2],
    para[3])[0]
print(time.time()-start)

lp: 0.001


rmse: 1.7232329977254979

lp: 0.9699285130813889


rmse: 2.5209948425847593

lp: 0.7678802590519337


rmse: 2.8062264189059887

lp: 0.1183621098921545


rmse: 2.8103699286368937

lp: 0.03489269816298368


rmse: 2.8103699286368937

lp: 0.5430034364904758


rmse: 2.8102410063198686

lp: 0.24204248785534918


rmse: 2.810286377504581

lp: 0.6908847980846642


rmse: 2.795980216150221

lp: 0.48447232329434387


rmse: 2.8039528650247556

lp: 0.310846911827551


rmse: 2.810192399894256

lp: 0.24543872789235172


rmse: 2.8103699286368937

lp: 1e-07


rmse: 2.3740423838179803

lp: 1.0


rmse: 2.8103699286368937

lp: 1e-07


rmse: 2.3531863293644983

lp: 0.41370796472562427


rmse: 1.7305523855510332

lp: 0.250915000492387


rmse: 1.722147401768716

lp: 1e-07


rmse: 1.6419877465816133

lp: 0.9099360599739807


rmse: 2.8103699286368937

lp: 0.9096059940514906


rmse: 1.6056177070681361

lp: 1.0


rmse: 1.62043263610419



E:\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


lp: 1.0


rmse: 2.063491850151649

lp: 1e-07


rmse: 1.5942840404691898

lp: 1e-07


rmse: 1.590073152082081



E:\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


lp: 1e-07


rmse: 1.6582190555297915



...

lp: 1e-07


rmse: 1.6598130242541855



E:\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


lp: 1e-07


rmse: 1.6090629240283454

lp: 1e-07


rmse: 1.7481456352065654

lp: 1e-07


rmse: 1.5324062792443982

lp: 0.9111611349133127


rmse: 2.8103699286368937

lp: 0.060716969587011775


rmse: 1.8547218208541396

lp: 0.9318363426002371


rmse: 1.577689650173878

lp: 0.982868796762069


rmse: 1.531964799051842

lp: 0.6257361622886641


rmse: 2.8103699286368937

lp: 0.25560382013991084


rmse: 1.5424706241733188

lp: 0.01467756516399364


rmse: 1.491753723699009

lp: 1e-07


rmse: 1.756027498269291

lp: 0.8071349326900046


rmse: 1.6926681776994437

lp: 0.6028440034704106


rmse: 1.770897608960137

lp: 0.7877087481017803


rmse: 2.8103699286368937

lp: 0.20831397852059455


rmse: 1.4547412224211125

lp: 0.179291084270894


rmse: 1.326274457113262

lp: 0.12572601557311106


rmse: 1.7627087685051985

lp: 0.28959283507162764


rmse: 2.7630880373618565

lp: 0.12081954646854733


rmse: 2.8103699286368937

lp: 0.8889220916948584


rmse: 1.8137785052651063

lp: 0.14332421573918364


rmse: 


rmse: 2.017359207866274

lp: 0.7807059572960712


rmse: 2.8103699286368937

lp: 0.9813165526842971


rmse: 2.8103699286368937

lp: 0.014820768956001299


rmse: 2.8103699286368937

lp: 0.94389154047689


rmse: 2.8103699286368937

lp: 0.6058091363968973


rmse: 1.6135816064823238

lp: 0.8644165852320757


rmse: 2.2171282699468065

lp: 0.2659517416783949


rmse: 2.8103699286368937

lp: 0.12184041696646981


rmse: 2.8103699286368937

lp: 0.10090481696522165


rmse: 2.5293377121531027

lp: 0.42584541891120725


rmse: 1.7508283421433757

lp: 0.6623469232240368


rmse: 1.6473943359947447

lp: 0.001


rmse: 2.1580218324179046

lp: 0.16802872122324192


rmse: 2.8060299454300446

lp: 0.5524776214786038


rmse: 2.8103699286368937

lp: 0.2456290837928058


rmse: 2.8103699286368937

lp: 0.6153692283853347


rmse: 2.8103699286368937

lp: 0.9838379043997636


rmse: 2.8103699286368937

lp: 0.900765132877854


rmse: 2.8103699286368937

lp: 0.10072100651799225


rmse: 2.8103699286368937

lp: 0.14957719


rmse: 2.139286611478114

lp: 0.769873780905829


rmse: 2.8103699286368937

lp: 0.7520055949307303


rmse: 2.3471754213229157

lp: 0.012983502736135777


rmse: 2.8103699286368937

lp: 0.832742554029276


rmse: 2.8103699286368937

lp: 0.06936340663941759


rmse: 2.646495942651062

lp: 0.0940497877122696


rmse: 2.8103699286368937

lp: 0.8073940101678466


rmse: 2.442420318807372

lp: 0.6529069540383192


rmse: 2.8103699286368937

lp: 0.15051780577382834


rmse: 2.8103699286368937

lp: 0.14755908376937973


rmse: 2.8103699286368937

lp: 0.39001778519333963


rmse: 2.8103699286368937

lp: 0.16548045214972582


rmse: 2.8103699286368937

lp: 1e-07


rmse: 2.2921578500986834

lp: 0.8630240464974817


rmse: 2.8103699286368937

lp: 0.04880675653864608


rmse: 2.8103699286368937

lp: 0.35115174325356974


rmse: 2.8103699286368937

lp: 0.19600280301928172


rmse: 2.8103699286368937

lp: 1e-07


rmse: 2.8103699286368937

lp: 0.057597603143158


rmse: 2.8103699286368937

lp: 0.6429321743530797


r

In [102]:
rmsev

array([[1.30489407, 1.52890883, 2.03130564, 1.56527476, 1.89826002],
       [1.90321035, 2.03272191, 2.00645296, 2.05535551, 2.01876271],
       [2.01333482, 1.73244291, 1.70971574, 1.85328676, 1.78291767]])

In [103]:
rmset

array([[2.14574204, 2.32894071, 2.16430186, 2.45480464, 2.49970011],
       [3.29087346, 2.47179972, 2.51401974, 2.58291168, 2.76337011],
       [2.5852393 , 2.14400908, 2.1002508 , 2.67315999, 2.19213115]])

In [80]:
pd.DataFrame(rmset).to_csv('rmsetdiff.csv')
pd.DataFrame(rmsev).to_csv('rmsevdiff.csv')